# Lammpsと使う

Juliaを使ってLammpsでの実験を効率化する方法について考えてみる

## Lammpsの出力データを処理する

- lammpsでは熱力学量をJSON形式で出力することができる
- Juliaには[JSON.jl](https://github.com/JuliaIO/JSON.jl)があってJSON形式をうまく処理してくれる
- この二つを組み合わせて効率よくデータを処理したい

まずは，JSON形式で物理量を出力するLAMMPSスクリプトを示す

```lammps
# 3d Lennard-Jones melt

units		lj
atom_style	atomic

lattice fcc 0.85
region		box block 0 10 0 10 0 10
create_box	1 box
create_atoms	1 box
mass		1 1.0

velocity	all create 3.0 87287 loop geom

pair_style	lj/cut 2.5
pair_coeff	1 1 1.0 1.0 2.5

neighbor	0.3 bin
neigh_modify	every 20 delay 0 check no

# JSONファイルで出力する
fix extra all print 50 """{"timestep": $(step),"vol":$(vol),"pxx": $(pxx), "pyy": $(pyy),"TotE":$(etotal)}""" title "" file therm.json screen no

fix 1 all nvt temp 1.0 1.0 $(dt*100)



thermo_modify	norm no
thermo		100
run		100000
```

これを実行するとカレントディレクトリに`therm.json`が生成される。
このファイルをJuliaで読み込む。

In [ ]:
using JSON #JSON形式を扱うパッケージ
using Plots

In [ ]:
# JSONファイルの最初の行に生成される謎の空行を削除する関数
function remove_blank_lines(file_path) 
    lines = readlines(file_path)
    if !isempty(lines) && isempty(strip(lines[1]))
        lines = lines[2:end]
    end
    open(file_path, "w") do file # Doブロック構文を使ったファイルの書き込み
        foreach(line -> write(file, "$line\n"), lines)
    end
end

In [ ]:
# lammpsの出力するjsonファイルから熱力学量の時系列データを読み出す関数
function readthermfile(filepath,thermname)
    cps=Float64[] # 要素の型を特に指定して配列を宣言
    open(filepath,"r") do file # doブロック構文を使ったファイルの読み書き,filemaneの関数を変数fileとして開く
        for line in eachline(file) # foreach関数を使わずfor文で書いてみる,eachlineは行ごとにひとまとめにしたベクトルを返す
            data = JSON.parse(line) # 読み込んだ行をJSON形式として解釈し，dict型の変数dataに代入
            p = data[thermname] # dict型変数からキーワードを使って目的の物理量を取り出す
            push!(cps,p) # それを配列に突っ込む
        end
    end
    return cps
end

In [ ]:
filepath="./data/therm.json" 
remove_blank_lines(filepath)
Es = readthermfile(filepath,"TotE")
plot(Es)

変数`Es`に[therm.json](./data/therm.json)の`TotE`の値を配列として読み込むことができた

## Lammpsのパラメタ変更を一括で行う

- lammpsでパラメタを少しづつ変えた計算をしたいとする
- たとえば温度を$0.5$から$15$まで$0.5$刻みに変化させたいとすると，素朴には30回パラメタを書き換えて計算を投げる必要がある
- このような操作を自動化するためにはよくbashが用いられる
- Juliaはスクリプト言語としての特徴を持っているので，Bashの代わりに用いるのにも向いている(要出典)
- 例として，等温系のシミュレーション[in.melt](./lammps/in.melt)の温度を$0.5$から$15$まで$0.5$刻みに変化させたシミュレーションを実行する方法を考えてみる

- まずは[in.template](./lammps/in.template)ファイルを作成する。
- このファイルでは，変化させたいパラメタ(シミュレーション温度)の部分を文字列`TEMPERATURE_PLACEHOLDER`にしてある
- Juliaを使ってこの文字列を値に置き換えたlammpsスクリプトファイルを生成し，そのファイルを元にしたジョブを投入する
- もっと良いやり方募集中


In [ ]:
templatepath="./lammps/in.template" # テンプレートファイルのパスを指定
temperatures=range(0.5,15,step=0.5) # 振りたいパラメタ(今回は温度)の配列を生成

for temp in temperatures
    template_script = read(templatepath,String) # ファイルをまるごと読み込む
    mod_script = replace(template_script,"TEMPERATURE_PLACEHOLDER"=>string(temp))
    fp=open("temp_script.in","w") 
    write(fp,mod_script)
    close(fp)
    run(`echo ここにT=$(temp)でのtemp_scripi.inを実行する文やジョブ投入の文を書きます`)
    #run(`mpirun -n 4 lmp_mpi -in temp_script.in`) # 例
    run(`rm temp_script.in`)
end

- 実際に用いるときは上記のようなコードを`subjob.jl`の名前で保存するなど，データ分析用のnoteookとは別にしておく方が良いかもしれない。(分析用のnotenookを実行する度に計算が走ると大変)
- 出力されたファイル(例えば＊.json)を一括で任意のディレクトリに移動させるような操作もJuliaでできる